In [1]:
from Models import *
from Utils import *
from FastModels import *
import warnings
# warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
from gensim.models import KeyedVectors
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('/work/data/bpe/en.wiki.bpe.op50000.model')
bpe = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)
bpe2 = KeyedVectors.load_word2vec_format("/work/data/bpe/en.wiki.bpe.op50000.d200.w2v.bin", binary=True)

In [3]:
bpe_dict = { bpe.index2word[i]: i for i in range(len(bpe.index2word))}

In [4]:
max_len = 10
enablePadding = True
nb_words= len(bpe_dict)
# run = KATE3D(len(bpe_dict), max_len, bpe.get_keras_embedding(True), [300,128], 2, "kcomp")
# run.initModel(sp, bpe_dict, bpe.index2word)

In [5]:
df_may, qrel_may = get_test_data("MayFlower", "/work/")
df_june, qrel_june = get_test_data("JuneFlower", "/work/")
df_july, qrel_july = get_test_data("JulyFlower", "/work/")

q_may = parse_texts_bpe(df_may.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_may = parse_texts_bpe(df_may.d.tolist(), sp, bpe_dict, max_len, enablePadding)

q_june = parse_texts_bpe(df_june.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_june = parse_texts_bpe(df_june.d.tolist(), sp, bpe_dict, max_len, enablePadding)

q_july = parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, enablePadding)
d_july = parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, enablePadding)

test_set = [[q_may, d_may, qrel_may, df_may, "MayFlower"], [q_june, d_june, qrel_june, df_june, "JuneFlower"], [q_july, d_july, qrel_july, df_july, "JulyFlower"]]


b'Skipping line 19898: expected 6 fields, saw 8\nSkipping line 20620: expected 6 fields, saw 8\nSkipping line 38039: expected 6 fields, saw 8\n'


In [6]:
def create_train_data(corpus, max_len, padding="pre"):
    x = []
    y = []
    for query in corpus:
        for j in range(len(query)-1):
            x.append(query[:j+1])
            y.append(query[j+1])
            
    x = pad_sequences(x, maxlen=max_len, padding=padding)
    y = np.array(y)
    return x, y

In [11]:
x_q_july, y_q_july = create_train_data(parse_texts_bpe(df_july.q.tolist(), sp, bpe_dict, max_len, False), max_len)
x_d_july, y_d_july = create_train_data(parse_texts_bpe(df_july.d.tolist(), sp, bpe_dict, max_len, False), max_len)
x_q_df, y_q_df = create_train_data(parse_texts_bpe(df.q.tolist(), sp, bpe_dict, max_len, False), max_len)

In [10]:
df = pd.read_csv("/work/data/train_data/QueryQueryLog", nrows=10000, names=["q", "d", "label"], sep="\t", header=None, error_bad_lines=False)

In [7]:
# _q_july = pad_sequences(pad_sequences(q_july, maxlen=max_len+1, padding='pre'), maxlen=max_len, truncating="post")
# _d_july = pad_sequences(pad_sequences(d_july, maxlen=max_len+1, padding='pre'), maxlen=max_len, truncating="post")

In [12]:
q_df = parse_texts_bpe(df.q.tolist(), sp, bpe_dict, max_len, enablePadding=enablePadding)
d_df = parse_texts_bpe(df.d.tolist(), sp, bpe_dict, max_len, enablePadding=enablePadding)
# q_df_one_hot = to_categorical(q_df, nb_words)
# q_df_one_hot = q_df_one_hot.reshape(len(q_df), max_len, nb_words)
# _q_df = pad_sequences(pad_sequences(q_df, maxlen=max_len+1, padding='pre'), maxlen=max_len, truncating="post")


In [13]:
q = q_july
x = to_categorical(q, nb_words)
x = x.reshape(len(q_july), max_len, nb_words)

d = d_july
y = to_categorical(d, nb_words)
y = y.reshape(len(d_july), max_len, nb_words)


In [36]:
run = QueryGenerator(nb_words, max_len, bpe.get_keras_embedding(train_embeddings=True), [300, 128], optimizer=RMSprop(lr=0.01))

In [13]:
_q_july = pad_sequences(pad_sequences(q_july, maxlen=max_len+1, padding='post'), maxlen=max_len, truncating="pre")
_d_july = pad_sequences(pad_sequences(d_july, maxlen=max_len+1, padding='post'), maxlen=max_len, truncating="pre")

In [14]:
_q_df = pad_sequences(pad_sequences(q_df, maxlen=max_len+1, padding='post'), maxlen=max_len, truncating="pre")

In [38]:
# x_train = np.concatenate([q_july, d_july])
# y_train = np.concatenate([np.expand_dims(q_july, axis=-1), np.expand_dims(d_july, axis=-1)])

x_train = q_july
y_train = np.expand_dims(q_july, axis=-1)
y_train = np.expand_dims(_q_july, axis=-1)

# y_train = np.concatenate([np.expand_dims(_q_july, axis=-1), np.expand_dims(_d_july, axis=-1)])

In [16]:
x_train = q_df
# y_train = np.expand_dims(q_df, axis=-1)
y_train = np.expand_dims(_q_df, axis=-1)
# y_train = q_df

In [14]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

w2v = Sequential()
w2v.add(bpe.get_keras_embedding(train_embeddings=True))
w2v.add(GlobalMaxPooling1D())

# pred = get_cosine_sim(w2v.predict(q_july), w2v.predict(d_july))
# print(auc(qrel_july, pred.flatten()))


def eval(epoch, logs):
    
    loss = logs.get("loss")
    val_loss = logs.get("val_loss")
    
#     may_ndcg, june_ndcg, july_auc = evaluate(run, cosine, test_set)
#     print_output = '%s, Epoch %d, [%.1f s], May = %.4f, June = %.4f, July = %.4f, Loss = %.4f, V_Loss = %.4f \n' % (run.name(), epoch, 0, may_ndcg, june_ndcg, july_auc, loss, val_loss)
#     print(print_output)
    
#     generate_output(run.model.predict(q_july), run.model.predict(d_july))

    if hasattr(run, 'generator'):
        generate_output(run.generator.predict(q_july), run.generator.predict(d_july))
    elif "seqvae" in run.name():

        generate_output(run.model.predict([q_july, _q_july]), run.model.predict([d_july, _d_july]))
    elif "kate_lstm" in run.name() or "vae_lstm" in run.name():
#         sample new query
#         generate_output(run.model.predict(q_july)[0], run.model.predict(d_july)[0])
        generate_output(run.decoder.predict(run.encoder.predict(q_july)), run.decoder.predict(run.encoder.predict(d_july)))
    elif "naive_vae" in run.name() or "nvdm" in run.name():
        print("no generated query")
    else:
        generate_output(run.model.predict(q_july), run.model.predict(d_july))
    
    
    #       combine with w2v
    if run.name() == "nvdm":
        q = run.encoder
#     q = np.concatenate([w2v.predict(q_july), run.encoder.predict(q_july)], axis=-1)
#     d = np.concatenate([w2v.predict(d_july), run.encoder.predict(d_july)], axis=-1)
    
    cosine = CosineSim(q.shape[-1])
    
    pred = cosine.model.predict([q, d])
    print(loss, val_loss, auc(qrel_july, pred.flatten()))
    
    
class QueryGenerator():
    
    def __init__(self, input_size, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=Adadelta(lr=2.), kl_weight=0, cos_weight=1):
        
        self.input_size = input_size
        self.dim = dim
        self.emb = emb
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std
        self.max_len = max_len
        self.nb_words = input_size
        self.optimizer = optimizer
        self.kl_weight = kl_weight
        self.cos_weight = cos_weight
        
        self.build()
    def name(self):
        return "lstm"
        
    def build(self):
        
        emb = self.emb
        emb.masking = True
        lstm = LSTM(self.dim[1], return_sequences=True)
        dense = Dense(self.nb_words)
        
        self.model = Sequential([emb, lstm, dense, Activation('softmax')])
        self.encoder = Sequential([emb, lstm, GlobalMaxPooling1D()])
        
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer=self.optimizer)
        
        
        

In [15]:
class KL_Annealing(Callback):
    
    def __init__(self, run):
        super(KL_Annealing, self).__init__()
        
        self.run = run
        self.kl_inc_rate = 1 / 5000. # set the annealing rate for KL loss
        self.cos_inc_rate = 1
        self.max_cos_weight = 150.
        self.max_kl_weight = 1.

    def on_epoch_end(self, epoch, logs=None):
        if hasattr(self.run, 'loss_layer'):
            self.run.loss_layer.kl_weight = min(self.run.loss_layer.kl_weight + self.kl_inc_rate, self.max_kl_weight)
        elif hasattr(self.run, "kl_weight"):
            self.run.kl_weight = min(self.run.kl_weight + self.kl_inc_rate, self.max_kl_weight)
            
            if "kate_lstm" in self.run.name() or "vae_lstm" in self.run.name():
                self.run.model.compile(optimizer=self.run.optimizer, loss=["sparse_categorical_crossentropy", self.run.kl_loss])
            else:
                self.run.model.compile(optimizer=self.run.optimizer, loss=[self.run.vae_loss])



In [16]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

class SeqVAE():
    
    def __init__(self, nb_words, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, mode=0, optimizer=Adadelta(lr=2.), enableGAN=False):
        
        
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std
        self.mode = mode
        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.enableGAN = enableGAN
        self.kl_weight = 0
        self.optimizer = optimizer
        
        self.build()
    def name(self):
        return "seqvae%d" % self.mode
    
    def sampling(self, args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.dim[0]), mean=0.,\
                                  stddev=self.epsilon_std)

        return z_mean + K.exp(z_log_var / 2) * epsilon
    
    def vae_loss(self, x, x_):
        
        rec_loss =  K.sum(K.sparse_categorical_crossentropy(x, x_), axis=-1)

        if self.mode == 0:
            kl_loss = - 0.5 * K.sum(1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var), axis=-1)
            return rec_loss + (self.kl_weight * kl_loss)
        elif self.mode == 1:
            kl_h_loss = - 0.5 * K.sum(1 + self.h_var - K.square(self.h_mean) - K.exp(self.h_var), axis=-1)
            kl_c_loss = - 0.5 * K.sum(1 + self.c_var - K.square(self.c_mean) - K.exp(self.c_var), axis=-1)
            return rec_loss + (self.kl_weight * (kl_h_loss + hl_c_loss))
    
    def build(self):
        
        encoder_inputs = Input(shape=(self.max_len,))
        encoder = LSTM(self.dim[0], return_state=True)
        encoder_outputs, state_h, state_c = encoder(self.emb(encoder_inputs))
        
        if self.mode == 0:
            self.z_mean = Dense(self.dim[0])(encoder_outputs)
            self.z_log_var = Dense(self.dim[0])(encoder_outputs)

            z = Lambda(self.sampling, output_shape=(self.dim[0],))([self.z_mean, self.z_log_var])
            encoder_states = [z, z]
        elif self.mode == 1:
            self.h_mean = Dense(self.dim[0])(state_h)
            self.h_var = Dense(self.dim[0])(state_h)
            
            self.c_mean = Dense(self.dim[0])(state_c)
            self.c_var = Dense(self.dim[0])(state_c)

            z_h = Lambda(self.sampling, output_shape=(self.dim[0],))([self.h_mean, self.h_var])
            z_c = Lambda(self.sampling, output_shape=(self.dim[0],))([self.c_mean, self.c_var])

            encoder_states = [z_h, z_c]
        
        # Set up the decoder, using `encoder_states` as initial state.
        decoder_inputs = Input(shape=(self.max_len,))
        # We set up our decoder to return full output sequences,
        # and to return internal states as well. We don't use the
        # return states in the training model, but we will use them in inference.
        decoder_lstm = LSTM(self.dim[0], return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(self.emb(decoder_inputs), initial_state=encoder_states)
        decoder_dense = Dense(self.nb_words, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)
        # Define the model that will turn
        # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
        self.model = Model([encoder_inputs, decoder_inputs], [decoder_outputs])
#         self.model.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy"])
        self.model.compile(optimizer=self.optimizer, loss=[self.vae_loss])

        self.encoder = Model(encoder_inputs, encoder_outputs)

In [57]:
run = SeqVAE(nb_words, max_len, bpe.get_keras_embedding(train_embeddings=True), [300, 128], mode=0, optimizer=RMSprop(lr=0.01))

In [40]:
# Test Data
_q_july = pad_sequences(pad_sequences(q_july, maxlen=max_len+1, padding='pre'), maxlen=max_len, truncating="post")
_d_july = pad_sequences(pad_sequences(d_july, maxlen=max_len+1, padding='pre'), maxlen=max_len, truncating="post")

x_train = [q_july, _q_july]
y_train = np.expand_dims(q_july, axis=-1)


In [45]:
run = VAE_LSTM(nb_words, max_len, bpe.get_keras_embedding(train_embeddings=True), [300, 128], kl_weight=0.2, optimizer=RMSprop(lr=0.01))

In [46]:
x_train = q_july
y_train = np.expand_dims(q_july, axis=-1)
x_train = q_df
y_train = np.expand_dims(q_df, axis=-1)

hist = run.model.fit(x_train, [y_train, y_train], epochs=50, verbose=0, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(verbose=1, patience=5),ReduceLROnPlateau(verbose=1, patience=3), LambdaCallback(on_epoch_end=eval), KL_Annealing(run), TQDMNotebookCallback()])

7968/|/[loss: 6.749, decoded_mean_loss_1: 4.696, decoded_mean_loss_2: 2.054] 100%|| 7968/8000 [00:49<00:00, 322.81it/s]h ives causes        : abbey voc hara hara hara hara hara hara hara activation

difference between whe y protein and whe y protein isolate : abbey fresno sharp sharp sharp sharp sharp sharp sharp activation

average cost of ky bella      : back activation activation activation activation activation activation activation activation activation

mac ular hole surgery recovery      : abbey abbey activation activation activation activation activation activation activation activation

kir stin powers        :          ww

top capital gains tax rate  2 0 <unk>  :   to to to to to to to to

 per oid ic table      :  mov mov mov mov mov mov mov mov mov

6.742331712722779 5.346508201599121 0.5740621316907203
h ives causes        : tar tar smoke smoke smoke smoke smoke tar tar ve

difference between whe y protein and whe y protein isolate : tar tar tar tar tar tar tar tar tar tar

KeyboardInterrupt: 

In [37]:
class VAE_LSTM(object):
    
    def __init__(self, nb_words, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=Adadelta(lr=2.), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight
        
        act = 'tanh'
        input_layer = Input(shape=(self.max_len,))
        self.embed_layer = emb
        self.embed_layer.mask_zero = True
        # self.embed_layer.trainable = False
#         bilstm = Bidirectional(LSTM(self.dim[0], name='lstm_1'))
        bilstm = LSTM(self.dim[0])


        hidden_layer1 = Dense(self.dim[0], kernel_initializer='glorot_normal', activation=act)
        
        h1 = self.embed_layer(input_layer)
        h1 = bilstm(h1)
        h1 = hidden_layer1(h1)

        self.z_mean = Dense(self.dim[1], kernel_initializer='glorot_normal')(h1)
        self.z_log_var = Dense(self.dim[1], kernel_initializer='glorot_normal')(h1)

        if self.comp_topk != None:
            self.z_mean_k = KCompetitive(self.comp_topk, self.ctype)(self.z_mean)
            encoded = Lambda(self.sampling, output_shape=(self.dim[1],))([self.z_mean_k, self.z_log_var])
        else:
            encoded = Lambda(self.sampling, output_shape=(self.dim[1],))([self.z_mean, self.z_log_var])


        # we instantiate these layers separately so as to reuse them later
        decoder_h = Dense(self.dim[0], kernel_initializer='glorot_normal', activation=act)
        # decoder_mean = Dense_tied(self.nb_words, activation='softmax', tied_to=hidden_layer1)
        decoder_mean = Dense(self.nb_words, activation='softmax')

        h_decoded = decoder_h(encoded)
        h_decoded = RepeatVector(self.max_len)(h_decoded)
        h_decoded = LSTM(self.dim[0], return_sequences=True, unroll=True)(h_decoded)
        x_decoded_mean = TimeDistributed(decoder_mean, name='decoded_mean')(h_decoded)

        if self.enableGAN:
            self.discriminator = self.build_discriminator()
            self.discriminator.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

            validity = self.discriminator(x_decoded_mean)

        if self.enableGAN:
            self.model = Model(outputs=[x_decoded_mean, validity], inputs=input_layer)
        else:
            self.model = Model(outputs=[x_decoded_mean, x_decoded_mean], inputs=input_layer)

        self.encoder = Model(outputs=self.z_mean, inputs=input_layer)

        # build a digit generator that can sample from the learned distribution
        decoder_input = Input(shape=(self.dim[1],))
        _h_decoded = decoder_h(decoder_input)
        _h_decoded = RepeatVector(self.max_len)(_h_decoded)
        _h_decoded = LSTM(self.dim[0], return_sequences=True)(_h_decoded)
        _x_decoded_mean = TimeDistributed(decoder_mean, name='decoded_mean')(_h_decoded)

        self.decoder = Model(outputs=_x_decoded_mean, inputs=decoder_input)



        if enableGAN:
            self.model.compile(optimizer=self.optimizer, loss=[self.vae_loss, "binary_crossentropy"])
        else:    
            self.model.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy", self.kl_loss])

    def name(self):
        return "kate_lstm_k%d" % self.comp_topk if self.ctype != None else "vae_lstm"
    def kl_loss(self, x, x_):

        kl_loss = - 0.5 * K.sum(1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var), axis=-1)

        return self.kl_weight * kl_loss



    def sampling(self, args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.dim[1]), mean=0.,\
                                  stddev=self.epsilon_std)

        return z_mean + K.exp(z_log_var / 2) * epsilon

256/|/[loss: 2.692, decoded_mean_loss_1: 2.692, decoded_mean_loss_2: 0.000]  75%|| 256/340 [00:17<00:03, 26.23it/s]

In [153]:
class QueryGenerator(object):
    
    def __init__(self, nb_words, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=Adadelta(lr=2.), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight
        
        act = 'tanh'
        input_layer = Input(shape=(self.max_len,))
        embedding = emb
        embedding.mask_zero = True
        
        # self.embed_layer.trainable = False
#         bilstm = Bidirectional(LSTM(self.dim[0], name='lstm_1'))
        bilstm = LSTM(self.dim[0])
        dense = Dense(self.nb_words, activation="sigmoid")
        
        pred = dense(bilstm(embedding(input_layer)))
        
        self.model = Model(input_layer, pred)
        # Dropout(0.2),
#         self.model = Sequential([embedding, bilstm,  dense])
        self.model.compile(optimizer=self.optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"])

In [159]:
run = QueryGenerator(nb_words, max_len, bpe.get_keras_embedding(True), [512,128], optimizer=RMSprop(lr=0.1))

In [160]:
df_tokens = set([])
for i in x_q_df:
    for j in i:
        df_tokens.add(j)

In [161]:
q_july_tokens = set([])
for i in x_q_july:
    for j in i:
        q_july_tokens.add(j)

d_july_tokens = set([])
for i in x_d_july:
    for j in i:
        d_july_tokens.add(j)

In [162]:
len(df_tokens), len(q_july_tokens.union(d_july_tokens)), len(df_tokens.intersection(q_july_tokens.union(d_july_tokens)))

(17490, 1271, 1140)

In [164]:
run.model.fit(x_q_df, y_q_df, validation_data=(x_q_july, y_q_july), batch_size=256, verbose=2, epochs=100, callbacks=[ReduceLROnPlateau(verbose=1, patience=3), TQDMNotebookCallback()])

Train on 361625 samples, validate on 1714 samples
Epoch 1/100
 - 223s - loss: 11.1880 - acc: 0.0338 - val_loss: 10.6010 - val_acc: 0.0093
Epoch 2/100
 - 221s - loss: 10.5422 - acc: 0.0206 - val_loss: 10.6337 - val_acc: 0.0111
Epoch 3/100
 - 215s - loss: 10.5542 - acc: 0.0200 - val_loss: 10.5873 - val_acc: 0.0187
Epoch 4/100
 - 216s - loss: 10.5103 - acc: 0.0259 - val_loss: 10.5857 - val_acc: 0.0158
Epoch 5/100
 - 218s - loss: 10.5080 - acc: 0.0259 - val_loss: 10.5923 - val_acc: 0.0152
Epoch 6/100
 - 217s - loss: 10.5147 - acc: 0.0251 - val_loss: 10.5781 - val_acc: 0.0158
Epoch 7/100
 - 215s - loss: 10.5061 - acc: 0.0260 - val_loss: 10.5913 - val_acc: 0.0134
Epoch 8/100
 - 217s - loss: 10.5052 - acc: 0.0262 - val_loss: 10.5793 - val_acc: 0.0152
Epoch 9/100
 - 214s - loss: 10.5019 - acc: 0.0265 - val_loss: 10.5745 - val_acc: 0.0146
Epoch 10/100
 - 216s - loss: 10.5031 - acc: 0.0264 - val_loss: 10.5835 - val_acc: 0.0146
Epoch 11/100
 - 215s - loss: 10.5033 - acc: 0.0262 - val_loss: 10.587

KeyboardInterrupt: 

In [99]:
def pred2query(pred, q, max_len, padding="pre"):
    x = []
    len_query = [len(i) for i in q]
    move = 0
#     print(len_query)
#     print(pred)
    for idx, i in enumerate(len_query):
#         print(idx, i, move)
        x.append([q[idx][0]] + list(pred[move: move+i-1]))
        move = move + i - 1

    x = pad_sequences(x, maxlen=max_len, padding=padding)
    return x

In [103]:
gen_q_july = pred2query(np.argmax(run.model.predict(x_q_july), axis=-1), q_july, max_len)
gen_d_july = pred2query(np.argmax(run.model.predict(x_d_july), axis=-1), d_july, max_len)

In [114]:
cosine = CosineSim(200)
pred = cosine.model.predict([w2v.predict(gen_q_july), w2v.predict(gen_d_july)])
auc(qrel_july, pred.flatten())

0.45255620538593255

In [26]:
class VAE_LSTM(object):
    
    def __init__(self, nb_words, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=Adadelta(lr=2.), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight
        
        act = 'tanh'
        input_layer = Input(shape=(self.max_len,))
        self.embed_layer = emb
#         self.embed_layer.mask_zero = True
        # self.embed_layer.trainable = False
#         bilstm = Bidirectional(LSTM(self.dim[0], name='lstm_1'))
        bilstm = LSTM(self.dim[0], return_sequences=True)


        hidden_layer1 = Dense(self.dim[0], activation=act)
        
        h1 = self.embed_layer(input_layer)
        h1 = bilstm(h1)
#         h1 = hidden_layer1(h1)

        self.z_mean = Dense(self.dim[1])(h1)
        self.z_log_var = Dense(self.dim[1])(h1)

        if self.comp_topk != None:
            self.z_mean_k = KCompetitive(self.comp_topk, self.ctype)(self.z_mean)
            encoded = Lambda(self.sampling, output_shape=(self.dim[1],))([self.z_mean_k, self.z_log_var])
        else:
            encoded = Lambda(self.sampling)([self.z_mean, self.z_log_var])


        # we instantiate these layers separately so as to reuse them later
        decoder_h = Dense(self.dim[0], activation=act)
        # decoder_mean = Dense_tied(self.nb_words, activation='softmax', tied_to=hidden_layer1)
        decoder_mean = Dense(self.nb_words, activation='softmax')

#         h_decoded = decoder_h(encoded)
#         h_decoded = RepeatVector(self.max_len)(h_decoded)
        h_decoded = LSTM(self.dim[0], return_sequences=True, unroll=True)(encoded)
        x_decoded_mean = TimeDistributed(decoder_mean, name='decoded_mean')(h_decoded)

        self.model = Model(outputs=[x_decoded_mean, x_decoded_mean], inputs=input_layer)

        self.encoder = Model(outputs=GlobalMaxPooling1D()(self.z_mean), inputs=input_layer)

        # build a digit generator that can sample from the learned distribution
        decoder_input = Input(shape=(self.dim[1],))
        _h_decoded = decoder_h(decoder_input)
        _h_decoded = RepeatVector(self.max_len)(_h_decoded)
        _h_decoded = LSTM(self.dim[0], return_sequences=True, unroll=True)(_h_decoded)
        _x_decoded_mean = TimeDistributed(decoder_mean, name='decoded_mean')(_h_decoded)

        self.decoder = Model(outputs=_x_decoded_mean, inputs=decoder_input)

 
        self.model.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy", self.kl_loss])

    def name(self):
        return "kate_lstm_k%d" % self.comp_topk if self.ctype != None else "vae_lstm"
    
    def kl_loss(self, x, x_):

        kl_loss = - 0.5 * K.sum(1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var), axis=-1)

        return self.kl_weight * kl_loss



    def sampling(self, args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.max_len, self.dim[1]), mean=0.,\
                                  stddev=self.epsilon_std)

        return z_mean + K.exp(z_log_var / 2) * epsilon

In [81]:
class GAN():
    def __init__(self, nb_words, max_len, emb, emb2, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=RMSprop(lr=0.01), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.emb2 = emb2
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight


        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer="adam",
            metrics=['accuracy'])
        
        self.discriminator.trainable = False


        # Build the generator
        self.generator, self.encoder = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.max_len,))
        text = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(text)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.model = Model(z, validity)
        self.model.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def build_generator(self):
        
        emb = self.emb
        print(emb)
        lstm = LSTM(self.dim[1], return_sequences=True)
        dense = Dense(self.nb_words)
        
        generator = Sequential([emb, lstm, dense, Activation('softmax')])
        encoder = Sequential([emb, lstm, GlobalMaxPooling1D()])
        
        generator.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy"])

        return generator, encoder

    def build_discriminator(self):
        
        input_layer = Input(shape=(self.max_len, self.nb_words,))
        emb = OnehotEmbedding(200, d=2)
        lstm = LSTM(self.dim[1])
        dense = Dense(1, activation="sigmoid")

        h = emb(input_layer)
        h = lstm(h)
        pred = dense(h)

        discriminator = Model(input_layer, pred)
        emb.set_weights([bpe2.vectors])

        return discriminator

    

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


In [107]:
run = GAN(nb_words, max_len, bpe.get_keras_embedding(train_embeddings=True), bpe2.vectors, [300, 128])

In [108]:
for epoch in range(5):
    run.generator.fit(d_july, np.expand_dims(d_july, axis=-1), verbose=2, epochs=1)
#     generate_output(run.generator.predict(d_july), run.generator.predict(d_july))
    print(gan_eval())

Epoch 1/1
 - 7s - loss: 6.0898
0.4463741688718895
Epoch 1/1
 - 1s - loss: 3.5941
0.45538037314594493
Epoch 1/1
 - 1s - loss: 2.5216
0.4832440125419734
Epoch 1/1
 - 0s - loss: 1.7184
0.47852965376148016
Epoch 1/1
 - 1s - loss: 1.0805
0.4709021770553047


In [109]:
generate_output(run.generator.predict(d_july), run.generator.predict(d_july))


TypeError: list indices must be integers or slices, not numpy.float32

In [112]:
# Adversarial ground truths
valid = np.ones((len(q_july), 1))
fake = np.zeros((len(q_july), 1))

for epoch in range(5):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    imgs = x

    # Generate a batch of new images
    gen_imgs = run.generator.predict(d_july)

    # Train the discriminator
    d_loss_real = run.discriminator.train_on_batch(imgs, valid)
    d_loss_fake = run.discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

#     noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

#     # Train the generator (to have the discriminator label samples as valid)
    g_loss = run.model.train_on_batch(d_july, valid)

#     # Plot the progress
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] AUC: %.4f" % (epoch, d_loss[0], 100*d_loss[1], g_loss, gan_eval()))
#     generate_output(run.generator.predict(q_july), run.generator.predict(d_july))
# For the combined model we will only train the generator
#     # If at save interval => save generated image samples
#     if epoch % sample_interval == 0:
#         self.sample_images(epoch)

0 [D loss: 0.748254, acc.: 49.77%] [G loss: 0.520398] AUC: 0.4191
1 [D loss: 0.709304, acc.: 49.77%] [G loss: 0.607252] AUC: 0.4221
2 [D loss: 0.686104, acc.: 49.77%] [G loss: 0.694397] AUC: 0.4150
3 [D loss: 0.677233, acc.: 82.75%] [G loss: 0.768257] AUC: 0.4199
4 [D loss: 0.719529, acc.: 35.33%] [G loss: 0.731035] AUC: 0.3982


In [104]:
def gan_eval():
    #       combine with w2v
    q = run.encoder.predict(q_july)
    d = run.encoder.predict(d_july)

    cosine = CosineSim(q.shape[-1])

    pred = cosine.model.predict([q, d])
    return auc(qrel_july, pred.flatten())
    

In [ ]:
class NaiveVAE():
    def __init__(self, nb_words, max_len, emb, emb2, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=RMSprop(lr=0.01), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.emb2 = emb2
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight


        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer="adam",
            metrics=['accuracy'])
        
        self.discriminator.trainable = False


        # Build the generator
        self.generator, self.encoder = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.max_len,))
        text = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(text)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.model = Model(z, validity)
        self.model.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def build_encoder(self):
        
        x = Input(shape=(self.max_len,))
        emb = self.emb
        dense = Dense(self.dim[0])
        mean = Dense(self.dim[1])
        var = Dense(self.dim[1])
        
        embed_x = emb(x)
        h = dense(embed_x)
        
        z_mean = mean(h)
        z_var = val(h)
        
        z = Lambda(self.sampling)([z_mean, z_var])
        
        encoder = Model(input_layer, z_mean)
        
        generator = Model(input_layer, z)

        return generator, encoder


    def build_decoder(self):
        
        input_layer = Input(shape=(self.dim[1],))
        
        dense = Dense(200 * self.max_len)
        
        h = Reshape((self.max_len, 200))(dense(input_lauer))
        
        

        
        
#         generator = Sequential([emb, lstm, dense, Activation('softmax')])
#         encoder = Sequential([emb, lstm, GlobalMaxPooling1D()])
        
#         generator.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy"])

        return generator, encoder

    def build_discriminator(self):
        
        input_layer = Input(shape=(self.max_len, self.nb_words,))
        emb = OnehotEmbedding(200, d=2)
        lstm = LSTM(self.dim[1])
        dense = Dense(1, activation="sigmoid")

        h = emb(input_layer)
        h = lstm(h)
        pred = dense(h)

        discriminator = Model(input_layer, pred)
        emb.set_weights([bpe2.vectors])

        return discriminator

    

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
    def sampling(self, args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.max_len, self.dim[1]), mean=0.,\
                                  stddev=self.epsilon_std)

        return z_mean + K.exp(z_log_var / 2) * epsilon


In [180]:
dim = [300, 128]

class NaiveVAE():
    def __init__(self):

        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], dim[1]), mean=0., stddev=1)

            return z_mean + K.exp(z_log_var / 2) * epsilon

        x = Input(shape=(max_len,))
        emb = bpe.get_keras_embedding(False)
        enc_dense = Dense(dim[0])
        mean = Dense(dim[1])
        var = Dense(dim[1])

        embed_x = GlobalMaxPooling1D()(emb(x))
        h = enc_dense(embed_x)

        z_mean = mean(h)
        z_var = var(h)

        z = Lambda(sampling)([z_mean, z_var])

        dec_dense = Dense(200 * max_len)

        _x = Reshape((max_len, 200))(dec_dense(z))
        _x = GlobalMaxPooling1D()(_x)

#         cos = merge([embed_x, _x], mode="mul")
        
#         pred = Dense(1, activation="sigmoid")(cos)
        
        # _x = dec_dense(z)
        


        
        def vae_loss(target, pred):
                rec_loss = objectives.cosine(embed_x, _x)
                kl_loss = - 0.5 * K.sum(1 + z_var - K.square(z_mean) - K.exp(z_var), axis=-1)

                return  rec_loss + (0.5 * kl_loss)


        self.model = Model(x, _x)
        self.model.compile(optimizer="adam", loss=vae_loss)
        self.encoder = Model(x, z)
    def name(self):
        return "naive_vae"

In [181]:
run = NaiveVAE()

In [182]:
x_train = q_july
y_train = np.expand_dims(q_july, axis=-1)
# x_train = q_df
# y_train = np.expand_dims(q_df, axis=-1)
# np.random.randint(2, size=(426,200))
hist = run.model.fit(x_train, np.ones(426) , epochs=20, verbose=0, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(verbose=1, patience=5),ReduceLROnPlateau(verbose=1, patience=3), LambdaCallback(on_epoch_end=eval), TQDMNotebookCallback()])

no generated query
3.8917165615979363 2.3599997232126637 0.5436634125731059
no generated query
1.7770575817893532 1.5273764965146086 0.5216704841112767
no generated query
1.0867487325387843 1.105464258859324 0.5146878961062065
no generated query
0.6749415769296534 0.80384520874467 0.5318997531632903
no generated query
0.3805125685299144 0.583930259527162 0.5216037714870244
no generated query
0.16747295470798718 0.4159471503523893 0.5624986101536614
no generated query
0.011721446365118026 0.2953929998153864 0.5424403477951478
no generated query
-0.10812742815298193 0.18251057315704433 0.5604527563432586
no generated query
-0.20703990599688363 0.09362420786258786 0.5429740487891659
no generated query
-0.2855706036090851 0.012799183123333509 0.4999888812292913
no generated query
-0.34797983379924996 -0.05801950871597889 0.5505792879539237
no generated query
-0.40131148240145514 -0.1114757994926253 0.5599635304320755
no generated query
-0.44407969818395726 -0.16168174702067709 0.5662789921

In [13]:
class NVDM():
    def __init__(self, mode=0):
        self.mode = mode
        
        
        dim = [300,200]
        X = Input(shape=(nb_words,))
        x = Input(shape=(max_len,))
        enc_emb = bpe.get_keras_embedding(True)
        h1 = Dense(dim[0], activation="relu")
        h2 = Dense(dim[0], activation="relu")
        mean = Dense(dim[1])
        log_var = Dense(dim[1])

        if self.mode == 0:
            z_mean = mean(h2(h1(X)))
            z_log_var = log_var(h2(h1(X)))
        elif self.mode == 1:
            z_mean = mean(h2(h1(enc_emb(x))))
            z_log_var = log_var(h2(h1(enc_emb(x))))


        def sampling(args):
            z_mean, z_log_var = args
            if self.mode == 0:
                epsilon = K.random_normal(shape=(K.shape(z_mean)[0], dim[1]), mean=0.,\
                                          stddev=1)
            elif self.mode == 1:
                epsilon = K.random_normal(shape=(K.shape(z_mean)[0], max_len, dim[1]), mean=0.,\
                                                          stddev=1)
            return z_mean + K.exp(z_log_var / 2) * epsilon

        
        if self.mode == 0:
            z = Lambda(sampling, output_shape=(dim[1],))([z_mean, z_log_var])
            repeated_z = RepeatVector(max_len)(z)
        elif self.mode == 1:
            z = Lambda(sampling, output_shape=(max_len, dim[1],))([z_mean, z_log_var])
            repeated_z = z

        # Decoder
        
        

        dec_emb = bpe2.get_keras_embedding(True)
        dec_bias = Embedding(nb_words, 200, embeddings_initializer="zeros")

        Rx = dec_emb(x)
        bx = dec_bias(x)

        _X = merge([merge([repeated_z, Rx], mode="mul"), bx], mode="sum")
        _X = Dense(nb_words, activation="softmax")(_X)
        
        if self.mode == 0:
            self.model = Model([X, x], [_X, _X])
            self.encoder = Model([X], z_mean)
        elif self.mode == 1:
            self.model = Model(x, [_X, _X])
            self.encoder = Model(x, GlobalMaxPooling1D()(z_mean))
            
        def kl_loss(X, _X):

        #     rec_loss = objectives.sparse_categorical_crossentropy(X, _X)
            kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

            return kl_loss

        self.model.compile(optimizer="adam", loss=["sparse_categorical_crossentropy", kl_loss])
        
    def name(self):
        
        return "nvdm%d" % self.mode


   

In [18]:
run = NVDM(mode=1)

In [15]:
X_q_july = np.zeros((len(q_july), nb_words))
for idx, i in enumerate(q_july):
    X_q_july[idx, i] = 1
x_q_july = q_july
x_q_july = np.expand_dims(x_q_july, axis=-1)

In [16]:
X_d_july = np.zeros((len(d_july), nb_words))
for idx, i in enumerate(d_july):
    X_d_july[idx, i] = 1
x_d_july = d_july
x_d_july = np.expand_dims(x_d_july, axis=-1)

In [28]:
X_df = np.zeros((len(q_df), nb_words))
for idx, i in enumerate(q_df):
    X_df[idx, i] = 1
x_df = q_df
y_df = np.expand_dims(x_df, axis=-1)

In [25]:
# x_df = np.concatenate([q_july, d_july])[:10000]
# y_df = np.concatenate([np.expand_dims(q_july, axis=-1), np.expand_dims(d_july, axis=-1)])[:10000]

In [29]:
len(x_df)

1000000

In [33]:
x_df[idx]

array([[ 1200,     3,  4639, ...,     0,     0,     0],
       [  713, 14240,  5032, ...,     0,     0,     0],
       [ 8562,  2589,  2572, ...,     0,     0,     0],
       ...,
       [ 4694,  3572,    13, ...,     0,     0,     0],
       [19731,  6128,     0, ...,     0,     0,     0],
       [  901,  3578,     0, ...,     0,     0,     0]], dtype=int32)

In [35]:
# mode = 0
# run.model.fit([X_df, x_df], [y_df, y_df], verbose=0, validation_split=0.1, batch_size=32, epochs=20, callbacks=[LambdaCallback(on_epoch_end=eval)])
# mode = 1
for i in range(10):
    idx = np.random.randint(len(x_df), size=10000)
    run.model.fit([x_df[idx]], [y_df[idx], y_df[idx]], verbose=0, validation_split=0.1, batch_size=32, epochs=1, callbacks=[TQDMNotebookCallback(), LambdaCallback(on_epoch_end=eval)])

no generated query
0.08481455051898956 0.08543515466153621 0.5099290622428785
no generated query
0.082918307059341 0.06969341568648815 0.49019324423491734
no generated query
0.08325616890523169 0.06623094818741083 0.4722475483110587
no generated query
0.08198393484784497 0.06756213203072547 0.5176566078854322
no generated query
0.08076373047629992 0.09768212014436722 0.5207698636838711
no generated query
0.0761272309521834 0.09563448145985604 0.49728701994707464
no generated query
0.07768686529414522 0.06402832823991776 0.49195001000689365
no generated query
0.07284428431259261 0.07934074947237968 0.4968422691187262
no generated query
0.07535207597414653 0.06112613801285625 0.4778514087482487
no generated query
0.07488689358863565 0.07653127740323544 0.5048700215704152


In [ ]:
for layer in model.layers:
    layer.trainable=False

In [22]:
def eval(epoch, logs):
    
    loss = logs.get("loss")
    val_loss = logs.get("val_loss")
    
#     may_ndcg, june_ndcg, july_auc = evaluate(run, cosine, test_set)
#     print_output = '%s, Epoch %d, [%.1f s], May = %.4f, June = %.4f, July = %.4f, Loss = %.4f, V_Loss = %.4f \n' % (run.name(), epoch, 0, may_ndcg, june_ndcg, july_auc, loss, val_loss)
#     print(print_output)
    
#     generate_output(run.model.predict(q_july), run.model.predict(d_july))

    if hasattr(run, 'generator'):
        generate_output(run.generator.predict(q_july), run.generator.predict(d_july))
    elif "seqvae" in run.name():

        generate_output(run.model.predict([q_july, _q_july]), run.model.predict([d_july, _d_july]))
    elif "kate_lstm" in run.name() or "vae_lstm" in run.name():
#         sample new query
#         generate_output(run.model.predict(q_july)[0], run.model.predict(d_july)[0])
        generate_output(run.decoder.predict(run.encoder.predict(q_july)), run.decoder.predict(run.encoder.predict(d_july)))
    elif "naive_vae" in run.name() or "nvdm" in run.name():
        print("no generated query")
    else:
        generate_output(run.model.predict(q_july), run.model.predict(d_july))
    
    
    #       combine with w2v
    if run.name() == "nvdm0":
        q = run.encoder.predict(X_q_july)
        d = run.encoder.predict(X_d_july)
    else:
        q = run.encoder.predict(q_july)
        d = run.encoder.predict(d_july)
#     q = np.concatenate([w2v.predict(q_july), run.encoder.predict(q_july)], axis=-1)
#     d = np.concatenate([w2v.predict(d_july), run.encoder.predict(d_july)], axis=-1)
    
    cosine = CosineSim(q.shape[-1])
    
    pred = cosine.model.predict([q, d])
    print(loss, val_loss, auc(qrel_july, pred.flatten()))

In [36]:
model = Sequential([bpe.get_keras_embedding(True), LSTM(300, return_sequences=True), Dense(nb_words, activation="softmax")])

In [45]:
model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(np.random.randint(nb_words, size=(10, 10)), np.random.randint(3, size=(10,10,nb_words)))

Epoch 1/10
10/10 [==============================] - 1s 104ms/step - loss: 10.8299
Epoch 2/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8291
Epoch 3/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8283
Epoch 4/10
10/10 [==============================] - 0s 8ms/step - loss: 10.8276
Epoch 5/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8268
Epoch 6/10
10/10 [==============================] - 0s 8ms/step - loss: 10.8259
Epoch 7/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8247
Epoch 8/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8234
Epoch 9/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8218
Epoch 10/10
10/10 [==============================] - 0s 9ms/step - loss: 10.8199


In [80]:
class VAE_LSTM(object):
    
    def __init__(self, nb_words, max_len, emb, dim, comp_topk=None, ctype=None, epsilon_std=1.0, optimizer=Adadelta(lr=2.), enableGAN=False, kl_weight=0):
        self.dim = dim
        self.comp_topk = comp_topk
        self.ctype = ctype
        self.epsilon_std = epsilon_std

        self.nb_words = nb_words
        self.max_len = max_len
        self.emb = emb
        self.enableGAN = enableGAN
        self.optimizer = optimizer
        self.kl_weight = kl_weight
        
        act = 'tanh'
        
        

        
        input_layer = Input(shape=(self.max_len,))
        mask = Masking(mask_value=0)
        self.embed_layer = emb
#         self.embed_layer = Embedding(nb_words, 200)
#         self.embed_layer.mask_zero = True
        # self.embed_layer.trainable = False
        bilstm = Bidirectional(LSTM(self.dim[0], return_sequences=True))
#         bilstm = LSTM(self.dim[0], return_sequences=True)


        hidden_layer1 = Dense(self.dim[0], kernel_initializer='glorot_normal', activation=act)
        
        h1 = self.embed_layer(mask(input_layer))
#         h1 = self.embed_layer(input_layer)

        h1 = bilstm(h1)
#         h1 = hidden_layer1(h1)

        self.z_mean = Dense(self.dim[1], kernel_initializer='glorot_normal')(h1)
        self.z_log_var = Dense(self.dim[1], kernel_initializer='glorot_normal')(h1)

        if self.comp_topk != None:
            self.z_mean_k = KCompetitive(self.comp_topk, self.ctype)(self.z_mean)
            encoded = Lambda(self.sampling, output_shape=(self.dim[1],))([self.z_mean_k, self.z_log_var])
        else:
            encoded = Lambda(self.sampling, output_shape=(self.max_len,self.dim[1],))([self.z_mean, self.z_log_var])


        # we instantiate these layers separately so as to reuse them later
#         decoder_h = Dense(self.dim[0], kernel_initializer='glorot_normal', activation=act)
        # decoder_mean = Dense_tied(self.nb_words, activation='softmax', tied_to=hidden_layer1)
        decoder_mean = Dense(self.nb_words, activation='softmax')

#         h_decoded = decoder_h(encoded)
#         h_decoded = RepeatVector(self.max_len)(h_decoded)
#         h_decoded = LSTM(self.dim[0], return_sequences=True, unroll=True)(encoded)    
        dec_lstm = Bidirectional(LSTM(self.dim[0], return_sequences=True))
    
        dec_hidden = TimeDistributed(decoder_mean, name='decoded_mean')
    
        h_decoded = dec_lstm(encoded)
        x_decoded_mean = dec_hidden(h_decoded)


        self.model = Model(outputs=[x_decoded_mean, x_decoded_mean], inputs=input_layer)

        self.encoder1 = Model(outputs=GlobalMaxPooling1D()(encoded), inputs=input_layer)
        self.encoder2 = Model(outputs=GlobalAveragePooling1D()(encoded), inputs=input_layer)
        self.encoder3 = Model(outputs=GlobalMaxPooling1D()(self.z_mean), inputs=input_layer)
        self.encoder4 = Model(outputs=GlobalAveragePooling1D()(self.z_mean), inputs=input_layer)


        # build a digit generator that can sample from the learned distribution
        decoder_input = Input(shape=(self.max_len, self.dim[1],))
        _h_decoded = dec_lstm(decoder_input)
        _x_decoded_mean = dec_hidden(_h_decoded)
#         _h_decoded = RepeatVector(self.max_len)(decoder_input)
#         _h_decoded = Bidirectional(LSTM(self.dim[0], return_sequences=True, unroll=True))(_h_decoded)
#         _x_decoded_mean = TimeDistributed(decoder_mean, name='decoded_mean')(_h_decoded)

        self.decoder = Model(outputs=_x_decoded_mean, inputs=decoder_input)



        if enableGAN:
            self.model.compile(optimizer=self.optimizer, loss=[self.vae_loss, "binary_crossentropy"])
        else:    
            self.model.compile(optimizer=self.optimizer, loss=["sparse_categorical_crossentropy", self.kl_loss])

    def name(self):
        return "kate_lstm_k%d" % self.comp_topk if self.ctype != None else "vae_lstm"
    def kl_loss(self, x, x_):

        kl_loss = - 0.5 * K.sum(1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var), axis=-1)

        return self.kl_weight * kl_loss



    def sampling(self, args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.max_len, self.dim[1]), mean=0.,\
                                  stddev=self.epsilon_std)

        return z_mean + K.exp(z_log_var / 2) * epsilon

In [81]:
run = VAE_LSTM(nb_words, max_len, bpe.get_keras_embedding(True), [300,128], optimizer="RMSProp", kl_weight=0.1)

In [82]:
q_df

array([[  449,   975,   221, ...,     0,     0,     0],
       [  449,   975,   221, ...,     0,     0,     0],
       [    0,   595, 18412, ...,     0,     0,     0],
       ...,
       [10850,   723,  2446, ...,     0,     0,     0],
       [ 6290,  4540,     0, ...,     0,     0,     0],
       [ 6290,  4540,     0, ...,     0,     0,     0]], dtype=int32)

In [ ]:
# for i in range(50):
run.model.fit(q_df[:10000], [np.expand_dims(q_df, axis=-1)[:10000], np.expand_dims(q_df, axis=-1)[:10000]], verbose=2, epochs=50)
#     generate_output(run.model.predict(q_july)[0], run.model.predict(d_july)[0])
    
#     generate_output(run.decoder.predict(run.encoder.predict(q_july)), run.deredict(run.encoder.predict(d_july)))
    


Epoch 1/50
 - 29s - loss: 4.1548 - decoded_mean_loss_1: 4.0342 - decoded_mean_loss_2: 0.1206
Epoch 2/50
 - 28s - loss: 3.4737 - decoded_mean_loss_1: 3.3227 - decoded_mean_loss_2: 0.1509
Epoch 3/50
 - 28s - loss: 3.2715 - decoded_mean_loss_1: 3.0871 - decoded_mean_loss_2: 0.1843
Epoch 4/50
 - 28s - loss: 3.1025 - decoded_mean_loss_1: 2.8877 - decoded_mean_loss_2: 0.2148
Epoch 5/50
 - 28s - loss: 2.9299 - decoded_mean_loss_1: 2.6905 - decoded_mean_loss_2: 0.2393
Epoch 6/50
 - 28s - loss: 2.7627 - decoded_mean_loss_1: 2.4905 - decoded_mean_loss_2: 0.2722
Epoch 7/50
 - 28s - loss: 2.5891 - decoded_mean_loss_1: 2.2843 - decoded_mean_loss_2: 0.3048
Epoch 8/50
 - 28s - loss: 2.4328 - decoded_mean_loss_1: 2.0996 - decoded_mean_loss_2: 0.3332
Epoch 9/50
 - 28s - loss: 2.2690 - decoded_mean_loss_1: 1.9049 - decoded_mean_loss_2: 0.3640
Epoch 10/50


In [65]:
def generate_output(x, y):

#     gen_x = np.argmax(generator.predict([_q_july, encoder.predict(x)]), axis=-1)
#     gen_y = np.argmax(generator.predict([_d_july, encoder.predict(y)]), axis=-1)
    
    gen_x = np.argmax(x, axis=-1)
    gen_y = np.argmax(y, axis=-1)

    train = []
    test = []
    count = 0
    for i, j, k, l in zip(gen_x, gen_y, q, d):
        gen_q = " ".join([bpe.index2word[t] for t in i])
        gen_d = " ".join([bpe.index2word[t] for t in j])
        real_q = " ".join([bpe.index2word[t] for t in k])
        real_d = " ".join([bpe.index2word[t] for t in l])

        train.append(sentence_bleu(real_q, gen_q))
        test.append(sentence_bleu(real_d, gen_d))
        
        
        real_q = real_q.replace("▁the", "")
        real_q = real_q.replace("▁","")
        gen_q = gen_q.replace("▁the", "")
        gen_q = gen_q.replace("▁","")
        
        
        
        print("%s : %s\n" % (real_q, gen_q))
#         if count > 5:
#             break
#         count = count + 1
#     print(np.mean(train), np.mean(test))

In [66]:
generate_output(run.model.predict(q_july)[0], run.model.predict(d_july)[0])


h ives causes        : samsung rict defence       

difference between whe y protein and whe y protein isolate : buddhist forge behavioral y ayed and compound at windows manual

average cost of ky bella      : pussy carav of ad stoke     

mac ular hole surgery recovery      : card cobras        

kir stin powers        : oil arm casino       

top capital gains tax rate  2 0 <unk>  : top gun emperors restaurants g  2 0 <unk> 

 per oid ic table      :  gy hinckley y bedroom     

warren buffett net worth  2 0 <unk>   : stimulating umo  stretch  2 0 <unk>  

how long spaghetti sauce in refriger ator    : how was wig wig in perth gles   

how do y make cyn der blocks    : how do after call rear take class   

rick ri ordan books       : bamb umo eme program      

colon cancer symptoms in women      : delaware fishing symptoms in      

jellyfish sting treatment        : pred motor ide       

american horror story season  <unk>     : sykes star loans season  <unk>    

act for america 

In [70]:
text = "1 2 3 4 5"
tmp = []
for t in sp.EncodeAsPieces(text):
    if not isinstance(t, str):
        t = str(t, "utf-8")

    if t in bpe_dict:
        tmp.append(bpe_dict[t])
    else:
        tmp.append(bpe_dict['<unk>'])

In [72]:
sp.EncodeAsPieces(text)

[b'\xe2\x96\x81',
 b'1',
 b'\xe2\x96\x81',
 b'2',
 b'\xe2\x96\x81',
 b'3',
 b'\xe2\x96\x81',
 b'4',
 b'\xe2\x96\x81',
 b'5']

In [73]:
tmp = np.array([tmp])

In [76]:
np.argmax(run.model.predict(tmp), axis=-1)

array([[[  167, 47368,   167,  5976,   167, 28623,   167, 50508,   167,
         50508]],

       [[  167, 47368,   167,  5976,   167, 28623,   167, 50508,   167,
         50508]]])

In [64]:
for i in range(2):
    for i in [run.encoder1, run.encoder2, run.encoder3, run.encoder4]:
        
        __q = i.predict(q_july)
        __d = i.predict(d_july)
        cosine = CosineSim(__q.shape[-1])
        pred = cosine.model.predict([__q, __d])
        print(auc(qrel_july, pred.flatten()))

0.4678333963397008
0.4858235673463942
0.45882719206564526
0.4267606573417243
0.4925837799372902
0.4986768662856634
0.45882719206564526
0.4267606573417243
